https://ratsgo.github.io/nlpbook/   
Do it!
BERT와 GPT로 배우는 자연어 처리    
트랜스포머 핵심원리와 허깅페이스 패키지 활용법   
이기창 지음

트랜스퍼 러닝이란 특정 태스크를 학습한 모델을 다른 태스크 수행에 재사용하는 기법을 가리킴  
BERT, GPT들에도 트랜스퍼 러닝이 적용됨    

업스트림 태스크는 다음 단어 맞히기, 빈칸 채우기 등 대규모 말뭉치의 문맥을 이해하는 과제   
다운스트림 태스크는 문서분류, 개체명 인식 등 우리가 풀고자하는 자연어 처리의 구체적인 문제   
GPT는 다음 단어 맞히기로 업스트림을 수행한 언어 모델(language model)
BERT는 빈칸 채우기로 업스트림을 수행한 마스크 언어 모델(masked language model)

업스트림 태스크는 수작업 없이도 다량의 학습 데이터를 아주 싼 값에 만들어 낼 수 있음   
이렇게 데이터 내에서 정답을 만들고 이를 바탕으로 모델을 학습하는 방법을 자기지도학습(self-supervised learning) 이라고 함

문서 분류 모델은 자연어(문서나 문장)를 입력받아 해당 입력이 어떤 범주(긍정, 중립, 부정)에 속하는지 확률값을 반환   
자연어 추론 모델은 문장 2개를 입력받아 두 문장 사이의 관계가 참, 거짓, 중립 등 어떤 범주인지 그 확률값을 반환   
개체명 인식 모델은 자연어(문서나 문장)를 입력받아 단어별로 기관명, 인명, 지명 등 어떤 개체명 범주에 속하느지 그 확률값을 반환   
질의응답 모델은 자연어(질문+지문)를 입력받아 각 단어가 정답의 시작일 확률값과 끝일 확률값을 반환
문장 생성 모델을 자연어(문장)를 입력받아 어휘전체에 대한 확률값을 반환

다운스트림 태스크의 종류

파인튜닝: 다운스트림 태스크 데이터 전체를 사용, 다운스트림 데이터에 맞게 모델 전체를 업데이트함   
프롬프트 튜닝: 다운스트림 태스크 데이터 전체를 사용, 다운스트림 데이터에 맞게 모델 일부만 업데이트   
인컨텍스트 러닝: 다운스트림 태스크 데이터의 일부만 사용, 모델을 업데이트하지 않음   
제로샷 러닝: 다운스트림 태스크 데이터를 전혀 사용하지 않음, 모델이 바로 다운스트림 태스크를 수행   
다운스트림 태스크 데이터를 1건만 사용, 모델은 1건의 데이터가 어떻게 수행되는지 참고한 뒤 다운스트림 태스크를 수행
퓨샷 러닝: 다운스트림 태스크 데이터를 몇 건만 사용, 모델은 몇 건의 데이터가 어떻게 수행되는지 참고한 뒤 다운스트림 태스크를 수행

학습 파이프라인   

각종 설정값 정하기, 데이터 내려받기, 프리트레인을 마친 모델 준비하기, 토크나이저 준비하기, 데이터로더 준비하기, 태스크 정의하기, 모델 학습하기

각종 설정값 정하기: 어떤 프리트레인 모델을 사용할지, 학습에 사용할 데이터는 무엇인지, 학습결과는 어디에 저장할지, 하이퍼파라미터(러닝 레이트 배치크기)

In [2]:
"""
from ratsnlp.nlpbook.classification import ClassificationTrainArguments

args = ClassificationTrainArguments(
    pretraained_model_name="beomi/kcbert-base",
    downstream_corpus_name="nsmc",
    downstream_corpus_root_dir="/root/Korpora",
    downstream_model_dir="gdrive/My Drive/nlpbook/checkpoint-doccls",
    learning_rate=5e-5,
    batch_size=32,
)
"""

'\nfrom ratsnlp.nlpbook.classification import ClassificationTrainArguments\n\nargs = ClassificationTrainArguments(\n    pretraained_model_name="beomi/kcbert-base",\n    downstream_corpus_name="nsmc",\n    downstream_corpus_root_dir="/root/Korpora",\n    downstream_model_dir="gdrive/My Drive/nlpbook/checkpoint-doccls",\n    learning_rate=5e-5,\n    batch_size=32,\n)\n'

데이터 내려받기   
파인튜닝하는 실습을 진행하기 위해 다운스트림 데이터를 미리 내려받아 둬야 함

In [3]:
"""
from Korpora import Korpora

Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=arge.sownstream_corpus_root_dir,
    force_download-True,
)
"""

'\nfrom Korpora import Korpora\n\nKorpora.fetch(\n    corpus_name=args.downstream_corpus_name,\n    root_dir=arge.sownstream_corpus_root_dir,\n    force_download-True,\n)\n'

프리트레인을 마친 모델 준비하기   
huggingface에서 만든 transformers라는 오픈소프 파이썬 패키지에 주목

In [4]:
"""
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=2,
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)
"""

'\nfrom transformers import BertConfig, BertForSequenceClassification\npretrained_model_config = BertConfig.from_pretrained(\n    args.pretrained_model_name,\n    num_labels=2,\n)\nmodel = BertForSequenceClassification.from_pretrained(\n    args.pretrained_model_name,\n    config=pretrained_model_config,\n)\n'

토크나이저 준비하기   
토큰(token)은 문장(sentence)보다 작은 단위이며, 한 문장은 여러 개의 토큰으로 구성되고, 토큰의 분리 기준은 그때그때 다를 수 있음.(띄어쓰기, 형태소, 등등)   
   
문장을 토큰 시퀀스(token sequence)로 분석하는 과정을 토큰화(tokenization), 토큰화를 수행하는 프로그램을 토크나이저(tokenizer)라고 함   
BPE(Byte Pair Encoding), 워드피스(wordpiece)알고리즘으로 실습

In [5]:
"""
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)
"""

'\nfrom transformers import BertTokenizer\ntokenizer=BertTokenizer.from_pretrained(\n    args.pretrained_model_name,\n    do_lower_case=False,\n)\n'

데이터 로더 준비하기   
데이터 로더는 데이터를 배치(batch) 단위로 모델에 밀어 넣어주는 역할을 함   
전체 데이터 가운데 일부 인스턴스를 뽑아(sample) 배치를 구성함   
데이터셋(dataset)은 데이터 로더의 구성요소 가운데 하나로 여러 인스턴스(문서+레이블)를 보유하고 있음   
토큰을 그에 해당하는 정수로 변환하는 과정을 인덱싱이라고 함-토크나이저가 토큰화와 함께 수행


In [6]:
"""
from torch.utils.data import DataLoader, RandomSampler

from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDatset

copus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
"""

'\nfrom torch.utils.data import DataLoader, RandomSampler\n\nfrom ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDatset\n\ncopus = NsmcCorpus()\ntrain_dataset = ClassificationDataset(\n    args=args,\n    corpus=corpus,\n    tokenizer=tokenizer,\n    mode="train",\n)\n'

태스크 정의   
파이토치 라이트닝(PyTorch Lightning)이라는 라이브러리 사용   
파이토치 라이트닝은 딥러닝 모델을 학습할 때 반복적인 내용을 대신 수행해줘 사용자가 모델 구축에만 신경쓸 수 있도록 돕는 라이브러리   
   

lightning 모듈을 상속받아 task 정의   
task에는 모델, 최적화 방법, 학습 과정등의 정의됨
최적화를 위해 옵티마이저, 러닝레이트 스케줄러 정의   
   

step은 
배치를 모델에 입력, 모델의 출력을 정답과 비교, 차이를 최소화하는 방향으로 모델을 업데이트
하는 순환과정으로 이루어짐
task는 1회 스텝에서 벌어지는 일들을 정의함

모델 학습하기   
   
트레이너는 파이토치 라이트닝에서 제공하는 객체로 실제 학습을 수행   
GPU등 하드웨어 설정, 학습기록 로깅, 체크포인트 저장 등 복잡한 설정들을 알아서 해줌

In [7]:
"""
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)
trainer = nlpbook.get_trainer(args)
trainer.fit(
    task,
    train_dataloader=train_dataloader,
)
"""

'\nfrom ratsnlp.nlpbook.classification import ClassificationTask\ntask = ClassificationTask(model, args)\ntrainer = nlpbook.get_trainer(args)\ntrainer.fit(\n    task,\n    train_dataloader=train_dataloader,\n)\n'

코랩(Colab)이란 Colaboratory의 준말로 구글에 서비스하는 가상 컴퓨팅 환경으로, 누구나 크롬등의 웹 브라우저에서 파이썬 코드를 작성하고 실행할 수 있음   
   
ctrl+enter를 누르면 실행결과가 출력됨
느낌표(!)를 입력한뒤 실행하면 해당 명령어가 대시셸에서 실행됨 

In [8]:
# 구글 드라이브와 연결하기   
# from google.colab import drive
# drive.mount("/gdrive", force_remount=True)

In [10]:
# 구글 드라이브에 파일 쓰기
# with open("/gdrive/My Drive/test.txt", "w") as f:
#     f.writelines("test")

퀴즈 풀며 정리하기

1. 자연어 처리 모델은 자연어를 입력받아서 해당 입력이 특정범주일 확률을 반환하는 확률함수다.
2. 모델출력을 정답에 가까워지도록 모델을 업데이트 하는 과정을 학습이라고 한다.
3. 트랜스퍼러닝이란 특정 태스크를 학습한 모델을 다른 태스크 수행에 재사용하는 기법이다.
4. 업스트림 태스크의 대표과제로 다음 단어 맞히기와 빈칸 단어 채우기가 있다.
5. 다운스트림 태스크는 자연어 처리의 구체적 과제들로, 보통 프리트레인을 마친 모델을 구조 변경없이 그대로 사용하거나 여기에 태스크 모듈을 덧붙인 형태로 수행한다.